### Importación de datos



In [ ]:
import plotly.express as px
import pandas as pd

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_1%20.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science-latam/refs/heads/main/base-de-datos-challenge1-latam/tienda_4.csv"

tienda = pd.read_csv(url)
tienda2 = pd.read_csv(url2)
tienda3 = pd.read_csv(url3)
tienda4 = pd.read_csv(url4)

In [41]:
tienda['Tienda'] = '1'
tienda2['Tienda'] = '2'
tienda3['Tienda'] = '3'
tienda4['Tienda'] = '4'
df = pd.concat([tienda, tienda2, tienda3, tienda4], ignore_index=True)
df.to_csv('base_completa.csv', index=False)


#1. Análisis de facturación



In [67]:

factura=df.groupby('Tienda')['Precio'].sum().reset_index( )
factura=factura.rename(columns={'Precio':'Precio_Miles'})
factura['Precio_Miles'] = factura['Precio_Miles'] / 1000
factura.sort_values(by='Precio_Miles', ascending=False)

,Tienda,Precio_Miles
0,1,1150880.4
1,2,1116343.5
2,3,1098019.6
3,4,1038375.7


In [74]:
Promedio = factura['Precio_Miles'].mean()
diferencia = (factura['Precio_Miles'][3] - Promedio) / Promedio * 100
diferencia = round(diferencia, 2)
diferencia


-5.68

# 2. Ventas por categoría

In [44]:
df.columns

Index(['Producto', 'Categoría del Producto', 'Precio', 'Costo de envío',
       'Fecha de Compra', 'Vendedor', 'Lugar de Compra', 'Calificación',
       'Método de pago', 'Cantidad de cuotas', 'lat', 'lon', 'Tienda'],
      dtype='object')

In [45]:
ventas_categoria=df.groupby(['Categoría del Producto','Tienda'])['Producto'].count().unstack().reset_index()
ventas_categoria.columns.name = None
ventas_categoria['Total']= ventas_categoria[['1','2','3','4']].sum(axis=1)
ventas_categoria.sort_values(by='Total', ascending=False, inplace=True)
ventas_categoria

,Categoría del Producto,1,2,3,4,Total
7,Muebles,465,442,499,480,1886
3,Electrónicos,448,422,451,451,1772
5,Juguetes,324,313,315,338,1290
2,Electrodomésticos,312,305,278,254,1149
1,Deportes y diversión,284,275,277,277,1113
4,Instrumentos musicales,182,224,177,170,753
6,Libros,173,197,185,187,742
0,Artículos para el hogar,171,181,177,201,730


In [46]:
ventas_categoria_px = df.groupby(['Categoría del Producto', 'Tienda'])['Producto'].count().sort_values().reset_index()
ventas_categoria_px.rename(columns={'Producto': 'Cantidad'}, inplace=True)
ventas_categoria_px = ventas_categoria_px.sort_values(by="Cantidad", ascending=True)

In [ ]:


fig = px.bar(
    ventas_categoria_px,
    x="Cantidad",
    y="Categoría del Producto",
    color="Tienda",
    barmode="group",
    orientation="h", 
    height=700,
    text='Cantidad',
    category_orders={
        "Tienda": ["1", "2", "3", "4"]  
    },
)
fig.update_traces(
    texttemplate='%{text}',
    textposition='outside',
    textfont_size=18,        
    textfont_color='black'
)

fig.update_layout(
    title="Cantidad de productos vendidos por categoría y tienda",
    xaxis_title="Cantidad",
    yaxis_title="Categoría del Producto"
)

fig.show()


In [48]:
ingresos_categoria_px = df.groupby(['Categoría del Producto', 'Tienda'])['Precio'].mean().reset_index()
ingresos_categoria_px.rename(columns={'Precio': 'Ingresos_Miles'}, inplace=True)
ingresos_categoria_px['Ingresos_Miles'] = (ingresos_categoria_px['Ingresos_Miles']/1000).round(2)

ingresos_categoria_px = ingresos_categoria_px.sort_values(by="Ingresos_Miles", ascending=True)

In [ ]:

fig = px.bar(
    ingresos_categoria_px,
    x="Ingresos_Miles",
    y="Categoría del Producto",
    color="Tienda",
    barmode="group",
    orientation="h", 
    height=700,
    text='Ingresos_Miles',
    category_orders={
        "Tienda": ["1", "2", "3", "4"]  
    },
)
fig.update_traces(
    texttemplate='%{text}',
    textposition='outside',
    textfont_size=18,        
    textfont_color='black'
)

fig.update_layout(
    title="Ingreso en miles promedio por categoría en cada tienda",
    xaxis_title="Ingreso Promedio (S/) en miles",
    yaxis_title="Categoría del Producto",
    xaxis=dict(
        showticklabels=False,        
        showgrid=False,              
        zeroline=False               
    ),
)

fig.show()


### Análsis

El análisis muestra que las categorías más demandadas son muebles, electrónicos y juguetes, ya que concentran la mayor cantidad de ventas. Sin embargo, al observar los ingresos, se evidencia que las categorías de electrónicos, electrodomésticos e instrumentos musicales son las que generan mayores ingresos. En este sentido, la tienda 4 presenta una ligera desventaja en la recaudación total, aunque mantiene volúmenes de venta similares a las demás tiendas en las categorías más populares. Esto sugiere que, a pesar de vender una cantidad comparable de productos, los precios o márgenes en tienda 4 podrían ser menores. Además, al comparar con la tienda 3, se aprecia una tendencia similar: buena rotación de productos, pero con ingresos más bajos.

# 3. Calificación promedio de la tienda


In [50]:
calificacion_tiendas= df[['Calificación','Tienda']].copy()
calificacion_tiendas['n'] = calificacion_tiendas.groupby('Tienda').cumcount()
tabla = pd.DataFrame(calificacion_tiendas.pivot(index='n', columns='Tienda', values='Calificación'))
tabla.mean().reset_index().rename(columns={0:'Calificación Promedio', 'Tienda':'Tienda'})

,Tienda,Calificación Promedio
0,1,3.976685
1,2,4.037304
2,3,4.048326
3,4,3.995759


In [ ]:
fig = px.box(calificacion_tiendas,
             x='Tienda',
             y='Calificación',
             color='Tienda',
             title='Distribución de calificaciones por tienda')

fig.update_traces(boxmean='sd')
fig.show()


### Análisis:

Se puede observar que las tiendas 1 y 4 presentan una mayor variación en las calificaciones, ya que sus cajas son más anchas y los valores se distribuyen de forma más dispersa, con una mediana cercana a 3.9. Esto indica que la satisfacción de los clientes en estas tiendas es más inconsistente y tiende a ser más baja en comparación con las demás. En contraste, las tiendas 2 y 3 muestran menor distribución de loss datos, concentrando la mayoría de sus calificaciones entre 3.5 y 5, lo que refleja opiniones más favorables. En general, la mediana de todas las tiendas se ubica alrededor de 4, indicando un nivel promedio de satisfacción moderadamente alto.

# 4. Productos más y menos vendidos

In [81]:
productos_vendidos = df.groupby(['Producto','Tienda']).agg(
    Cantidad_Vendida=('Producto', 'count'),
    Suma_Precios_Miles=('Precio', 'sum'),
).reset_index()
productos_vendidos['Suma_Precios_Miles'] = productos_vendidos['Suma_Precios_Miles'] / 1000
productos_vendidos


,Producto,Tienda,Cantidad_Vendida,Suma_Precios_Miles
0,Ajedrez de madera,1,49,1430.6
1,Ajedrez de madera,2,45,1287.7
2,Ajedrez de madera,3,47,1355.3
3,Ajedrez de madera,4,39,1075.5
4,Armario,1,60,29462.3
...,...,...,...,...
199,Tablet ABXY,4,45,56723.1
200,Vaso térmico,1,54,2463.4
201,Vaso térmico,2,47,2155.9
202,Vaso térmico,3,38,1725.6


In [88]:
productos_mas_vendidos = productos_vendidos.groupby(['Producto'], observed=False).agg(
    Cantidad_Vendida_Total=('Cantidad_Vendida', 'sum'),
    Suma_Precios_Miles=('Suma_Precios_Miles', 'sum'),
    
).reset_index()
productos_mas_vendidos_top5=productos_mas_vendidos. sort_values(by='Cantidad_Vendida_Total',ascending=False).head(5)
productos_mas_vendidos_top5

,Producto,Cantidad_Vendida_Total,Suma_Precios_Miles
32,Mesa de noche,210,58419.5
12,Carrito de control remoto,206,14767.7
33,Microondas,206,115699.5
7,Batería,203,147806.3
11,Cama king,201,135780.4


In [98]:
productos_mas_vendidos_list=productos_mas_vendidos_top5['Producto'].unique()
productos_mas_vendidos_gf=productos_vendidos.query("@productos_mas_vendidos_list in Producto")
productos_mas_vendidos_gf=productos_mas_vendidos_gf.sort_values(by='Cantidad_Vendida', ascending=True)


In [ ]:

fig = px.bar(
    productos_mas_vendidos_gf,
    x='Producto',
    y='Cantidad_Vendida',
    barmode='group',
    color='Tienda',
    title='Productos más vendidos por tienda ',
    category_orders={
        "Tienda": ["1", "2", "3", "4"]  
    },
)

fig.update_layout(
    xaxis_title='Producto',
    yaxis_title='Cantidad Vendida',
)

fig.show()


In [ ]:
productos_menos_vendidos=productos_vendidos.groupby('Producto', observed=False).agg(
    Cantidad_Vendida_Total=('Cantidad_Vendida', 'sum'),
    Suma_ingresos_miles=('Suma_Precios_Miles', 'sum')).reset_index().sort_values(by='Cantidad_Vendida_Total', ascending=True)

productos_menos_vendidos=productos_menos_vendidos.head(5)
productos_menos_vendidos

,Producto,Cantidad_Vendida_Total,Suma_ingresos_miles
13,Celular ABXY,157,157911.1
4,Auriculares con micrófono,158,22748.5
34,Mochila,163,16065.7
22,Guitarra eléctrica,165,129404.8
14,Ciencia de datos con Python,166,9036.1


In [96]:
productos_menos_vendidos_list=productos_menos_vendidos['Producto'].unique()
productos_menos_vendidos_gf=productos_vendidos.query("@productos_menos_vendidos_list in Producto")
productos_menos_vendidos_gf=productos_menos_vendidos_gf.sort_values(by='Cantidad_Vendida', ascending=False)

In [97]:

fig = px.bar(
    productos_menos_vendidos_gf,
    x='Producto',
    y='Cantidad_Vendida',
    barmode='group',
    color='Tienda',
    title='Productos menos vendidos por  tienda ',
    category_orders={
        "Tienda": ["1", "2", "3", "4"]  
    },
)

fig.update_layout(
    xaxis_title='Producto',
    yaxis_title='Cantidad Vendida',
)

fig.show()


# 5. Envío promedio por tienda

In [59]:
envio_por_tienda=df.groupby('Tienda')['Costo de envío'].mean().reset_index()
envio_por_tienda

,Tienda,Costo de envío
0,1,26018.609580
1,2,25216.235693
2,3,24805.680373
3,4,23459.457167


In [60]:
georeferencia=df.groupby(['lat','lon']).agg(
    Ubicaciones=('Lugar de Compra', 'first'),
    Total_Compras=('Producto', 'count'),
    Calificación_Media=('Calificación', 'mean'),
    Costo_Envio_Medio=('Costo de envío', 'mean'),
    Ingresos_Totales=('Precio', 'sum'),
    Producto_mejor_vendida=('Producto', lambda x: x.mode()[0] if not x.mode().empty else None),  
    Categoria_mas_vendida=('Categoría del Producto', lambda x: x.mode()[0] if not x.mode().empty else None),
    Tienda_mas_frecuente=('Tienda', lambda x: x.mode()[0] if not x.mode().empty else None  )
).reset_index()


In [ ]:

mercados_geo = (
    georeferencia.groupby('Tienda_mas_frecuente')['Total_Compras']
    .count()
    .reset_index(name='Cantidad_Compras') )

fig = px.pie(
    mercados_geo,
    names='Tienda_mas_frecuente',  
    values='Cantidad_Compras',    
    title='Distribución de compras por tienda',
)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()


In [ ]:
fig = px.scatter_map(georeferencia, lat='lat', lon='lon', color='Ingresos_Totales',
                     
                     title='Mapa de ingresos totales por ubicación',
                     size='Ingresos_Totales',
                     text= 'Ubicaciones',
                     hover_data={'lat':False, 'lon':False, 
                                 'Ingresos_Totales':':',
                                 'Total_Compras':':',
                                 'Calificación_Media':':.2f',
                                 'Costo_Envio_Medio':':.2f',
                                 'Producto_mejor_vendida':True,
                                 'Categoria_mas_vendida':True,
                                 'Tienda_mas_frecuente':True
                                 },
                     hover_name='Ubicaciones',
                     
                  color_continuous_scale=px.colors.cyclical.Edge, size_max=30, zoom=4)
fig.show()

## Informe Final de Análisis y Recomendación

Con base en los análisis realizados y las visualizaciones generadas, se ha evaluado el desempeño de las diferentes tiendas considerando los siguientes factores: ingresos totales, categorías de productos, calificaciones promedio de los clientes, productos más y menos vendidos, y coste de envío promedio. A continuación, se presenta un resumen de los hallazgos y la recomendación final para el Sr. Juan.

### 1. Ingresos totales de las tiendas

Los resultados muestran que la Tienda 1 es la que obtiene los mayores ingresos totales, mientras que la Tienda 4 presenta los valores más bajos. El promedio general de ingresos es de aproximadamente 1,100,904.8, destacando que la Tienda 4 se encuentra más lejos de este promedio aproximadamente un 6% menor. Esto sugiere que, aunque la Tienda 1 lidera en recaudación, existen diferencias notables que deben analizarse considerando otros factores como costos y satisfacción del cliente.

### 2. Categorías de productos más y menos vendidas

El análisis de categorías evidencia que la Tienda 4 presenta un mayor volumen de ventas en cantidad de productos, aunque no necesariamente los mayores ingresos. Esto indica que su estrategia se enfoca más en la cantidad de ventas que en el margen de ganancia por producto, mientras que otras tiendas, como la Tienda 1, logran mejores resultados económicos con menos unidades vendidas.

### 3. Calificaciones promedio de los clientes

En cuanto a la percepción del cliente, se observa que las Tiendas 2 y 3 presentan mejores calificaciones promedio, ubicándose en los cuartiles superiores, lo que representa una fortaleza en términos de satisfacción y reputación. En contraste, las Tiendas 1 y 4 registran calificaciones más bajas, situadas en el cuartil inferior, lo que refleja posibles debilidades en la experiencia del cliente o en la calidad del servicio.

### 4. Productos más y menos vendidos

Entre los productos más vendidos destacan la mesa de noche, el carrito a control remoto y el microondas, siendo estos los que generan mayores ingresos. La Tienda 1 sobresale con la mayor cantidad de unidades vendidas de estos productos, mientras que la Tienda 4 muestra menor desempeño en ventas. En cuanto a los productos menos demandados, las Tiendas 2 y 4 concentran la mayor frecuencia de ventas bajas, lo que podría indicar una oferta de productos menos atractiva o menos alineada con la demanda del mercado.

### 5. Coste de envío promedio

Respecto al coste de envío, la Tienda 4 presenta el promedio más bajo, lo cual es una ventaja competitiva que puede atraer más clientes sensibles al precio total de compra como se observa en el mapa de georeferencia la tienda maas frecuente en Bogota es la tienda 4. Por otro lado, la Tienda 1 tiene el coste de envío más alto, lo cual podría afectar negativamente la decisión de compra, a pesar de sus altos ingresos actuales.

# Conclusión y Recomendación

En función del análisis integral, se proponen dos alternativas estratégicas para el Sr. Juan:

Tras el análisis de ingresos, ventas, calificaciones y costos de envío, se concluye que la Tienda 1 es la opción más conveniente para vender. Actualmente presenta los mayores ingresos y volumen de ventas, lo que la convierte en el activo con mayor valor de mercado. Sin embargo, sus altos costos de envío y menor satisfacción del cliente podrían afectar su rentabilidad futura si no se corrigen.

Venderla en este punto permitiría obtener un retorno óptimo, aprovechando su fortaleza financiera antes de que los factores de riesgo impacten su desempeño. En contraste, la Tienda 4 presenta menores costos de envio, lo que ofrece una oportunidad para fortalecerla y captar un segmento más sensible al precio y recapturar parcialmente el mercado que deja la tienda 1, y ademas en Boston es la más frecuente. 

En conclusión, se recomienda vender la Tienda 1 y potenciar la Tienda 4 como alternativa de crecimiento sostenible.